In [1]:
import pymatgen.core as mg

In [27]:
API_KEY = '你的APIKEY'

In [28]:
import itertools
import IPython
import random
import numpy as np

In [29]:
all_symbols = ["H", "Li", "Be","C", "N", "O", "Na", 
            "Mg", "Al", "Si", "P", "S", "K", "Ca", "Sc", "Ti",
            "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge", "As",
            "Se", "Br", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc", "Ru", 
            "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "Cs", 
            "Ba", "La", "Ta", "W", "Re", "Os"]
complx = list(itertools.combinations(all_symbols, 2)) 

complx_picked=random.sample(complx,300)

In [30]:
system=list(map(lambda x: complx_picked[x][0] + '-' + complx_picked[x][1],np.arange(len(complx_picked))))

In [77]:
from pymatgen.ext.matproj import MPRester
def que(x):
    a = MPRester(API_KEY)
    data = a.query(criteria=x,properties=["unit_cell_formula", "pretty_formula","spacegroup"])
    return data

300

In [78]:
data=[]
valid_num=[]
for i in system:
    res=que(i)
    if res !=[]:
        data.extend(res)
        valid_num.append(system.index(i))
        print(len(res),"hello，jojo!",system.index(i))

15 hello，jojo! 0
5 hello，jojo! 1
13 hello，jojo! 2
3 hello，jojo! 3
2 hello，jojo! 4
15 hello，jojo! 5
7 hello，jojo! 6
6 hello，jojo! 7
15 hello，jojo! 8
3 hello，jojo! 9
5 hello，jojo! 10
5 hello，jojo! 11
67 hello，jojo! 12
7 hello，jojo! 13
5 hello，jojo! 14
20 hello，jojo! 15
11 hello，jojo! 17
7 hello，jojo! 18
116 hello，jojo! 19
1 hello，jojo! 20
1 hello，jojo! 22
6 hello，jojo! 24
3 hello，jojo! 25
11 hello，jojo! 26
2 hello，jojo! 27
10 hello，jojo! 28
6 hello，jojo! 29
12 hello，jojo! 30
6 hello，jojo! 32
9 hello，jojo! 34
8 hello，jojo! 35


KeyboardInterrupt: 

In [84]:
len(data)

402

In [49]:
length=np.arange(len(data))
formula=list(map(lambda x:data[x]['pretty_formula'],length))

In [86]:
import pymatgen.core as mg

In [87]:
def mat_descriptor_calculation(material):
    mat=mg.Composition(material)
    fraction_list=list(map(lambda x: mat.get_atomic_fraction(x), mat))
    sort_frac=fraction_list.sort(reverse=True)
    if sort_frac is not None:
        fraction=sort_frac[0]/sort_frac[1]
    else:
        fraction=1
    atomic_number=list(map(lambda x: x.Z, mat))
    electronegativity=list(map(lambda x: x.X, mat))
    element_group=list(map(lambda x: x.group, mat))
    descriptor=atomic_number+electronegativity+element_group
    descriptor.append(fraction)
    return descriptor

mat_descriptor=list(map(lambda x:mat_descriptor_calculation(data[x]['pretty_formula']),length))

In [90]:
crystal_system=list(map(lambda x:data[x]['spacegroup']['crystal_system'],length))
crystal=list(set(crystal_system))

In [91]:
len(crystal)

7

In [93]:
label=[]
for i in crystal_system:
    for j in np.arange(len(crystal)):
        if i==crystal[j]:
            i=j
            label.append(i)

In [94]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

steps = [
    ('autoscale', StandardScaler()),
    ('NB', GaussianNB())]

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mat_descriptor, 
                                                    label, test_size=0.1, random_state=33)

pipeline = Pipeline(steps=steps)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.2926829268292683